# A Notebook to use Decision Tree Classifiers

This notebook shows how to train a decision tree to classify unseen instances.

For those of you interested in understanding the code, it uses predefined functions from the sklearn library of machine learning primitives and from the graphviz library to generate visualizations. A few more details about the code:
* The variable "dataset" stores the name of text file that you input and is passed as an argument of the function "loadDataSet()".
* The variable "attributes" stores the names of all features. The variable "instances" stores the values of all features in the training set. The variable "labels" stores the labels of all instances.
* The variable "clf" stores a decision tree model, and it can be trained with "instances" and "labels". Once the model is trained, it can be used to predict unseen instances. We use a type of decision tree algorithm called CART (Classification and Regression Trees).
The variable "n_foldCV" stores the number of times of n-fold cross validation that you input.
* The function "cross_val_scores" assesses the accuracy scores of a decision tree model. Its inputs are "clf", "instances", "labels", "n_foldCV".
* The variable "scores" stores the accuracy of an n-fold cross validation of the model.

The first step is to get the data. Here we use bash command within the notebook (preceded by `!`) to download the data.

In [ ]:
!wget https://raw.githubusercontent.com/khider/INF549/master/Homework%20Assignments/Homework%204/Dataset/lenses.csv
!wget https://raw.githubusercontent.com/khider/INF549/master/Homework%20Assignments/Homework%204/Dataset/iris.csv

Next, we will be importing packages and defining a few custom functions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import graphviz
import matplotlib.pyplot as plt
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

def plot_cv_indices(cv, X, y, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0
        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
              c=y, marker='_', lw=lw, cmap=cmap_data)
    
    # Formatting
    yticklabels = list(range(n_splits)) + ['class']
    ax.set(yticks=np.arange(n_splits+1) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(X)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

def loadDataSet(dataset):
    with open(dataset) as f:
        data=f.readlines()
        attributes=data[0].rstrip().split(',')[:-1]
        instances=[entry.rstrip().split(',')[:-1] for entry in data[1:]]
        dataArray=[]
        for i in range(len(instances[0])):
            dataArray.append([float(instance[i]) for instance in instances])
        instances=np.array(dataArray).T
        labels=[entry.rstrip().split(',')[-1] for entry in data[1:]]
        return attributes,instances,labels

## Training: Building a Decision Tree Classifier

The cell below asks for a dataset. It trains a decision tree classifier.
We provide two classification datasets that could be applied to the decision tree algorithms.
* "iris.csv" has four attributes with continuous values describing three different iris species.
* "lenses.csv" contains four attributes with discrete values and three classes.

Before training your classifier, run the cell below to take a look at the dataset.

In [ ]:
dataset=input('Please Enter Your Dataset:')
df=pd.read_csv(dataset)
display(df)

The following cell gives the number of instances in all distinct classes.

Label Name is the name of the last column in the dataframe.

In [ ]:
label_name = input('Enter The Label Name:')
print(df[label_name].value_counts())

Before we run the following cell, let's learn an important concept called feature encoding. Many classifiers only take numerical data and some datasets have features that are not numerical. For example, a feature can be the state that a person lives in. Those are called [categorical features](https://en.wikipedia.org/wiki/Categorical_variable). In that case,we need to encode categorical features into discrete values. This process is called feature encoding.

In our notebook, if your dataset contains categorical features, you will see the code rules in the cell below. In the next section, when you are prompted to input test set for prediction, the algorithm will automatically encode the relevant categorical features according to the code rules shown below.

In [ ]:
attributes,instances,labels=loadDataSet(dataset)
clf = tree.DecisionTreeClassifier()
clf.fit(instances,labels)

## Visualizing a Decision Tree

The following cell will generate a visualization of the decision tree.

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,max_depth=5,\
feature_names=attributes,class_names=clf.classes_,label='all',\
filled=True,special_characters=True) 
graph = graphviz.Source(dot_data) 
graph

## Evaluating the Accuracy of a Decision Tree Classifier¶

The following cell will run cross-validation to evaluate your decision tree classifier.

K Fold Cross-Validation is used to estimate prediction error. The dataset is randomly divided into K folds. The first fold acts as the validation set while the method is fit on remaining K-1 folds. Mean Squared Error is calculated on the observations from the held-out fold. The process is repeated K times, taking a different part each time.

![Cross-validation](https://miro.medium.com/max/2000/1*fW_qNKmvmg8duWoQnp_PoQ.png)

In [ ]:
n_foldCV=int(input("Please Enter the Number of Folds:"))
attributes,instances,labels=loadDataSet(dataset)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(instances,labels)
scores = cross_val_score(clf, instances, labels, cv=n_foldCV)

The following cell will output the accuracy score for each fold and the accuracy estimate of the model under 95% confidence interval.

In [ ]:
print("Scores:")
[print(score) for score in scores]
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Prediction: Classifying New Instances Using a Decision Tree Classifier

The cell below classifies new instances with the decision tree you created.

When you are prompted to input a prediction set, please create an example of an instance that looks like the instances in the training set.

In [ ]:
testset=input('Please Enter Your Prediction Set:')
testset=testset.strip().split(",")
temp=[]
for i in range(len(testset)):
        temp.append(float(testset[i]))
testset=np.array(temp).reshape((1,len(temp)))
predictions=clf.predict(testset)
print(predictions)